In [4]:
from src.pyAOM_utils import *

ModuleNotFoundError: No module named 'aom_overlap'

### GTO to STO ORBITAL PROJECTION

In [ ]:
# STO parameters for GTO-STO projection step, refers to exponential coefficients of STO basis functions
STO_proj_dict={
    'H':{'STOs':1  ,'1s':1.0000},
    'C':{'STOs':1+3,'2s':1.6083,'2p':1.442657},
    'N':{'STOs':1+3,'2s':1.9237,'2p':1.646703},
    'O':{'STOs':1+3,'2s':2.2458,'2p':1.858823},
    'F':{'STOs':1+3,'2s':2.5638,'2p':2.136394},
    'S':{'STOs':1+3,'3s':2.1223,'3p':1.651749},
}

# general configuration
config={
    'label': '6T',
#    'single_mol_xyz': 'single_molecules/6T_opt_geom_EnGn.xyz',
    'single_mol_xyz': 'single_molecules/6T_opt_geom_EnGn.xyz',
    'basis': 'DZVP-GTH',
    'potential': 'GTH-PBE',
    'MO': 73,
    'cp2k_template': 'templates/sp_GGA_template.inp',
    'cp2k_basis_file': 'cp2k_files/GTH_BASIS_SETS',
}
# CP2K DFT parameters
CP2K_QS_GGA_parameters={
    'PROJECT_NAME': '6T',
    'XC_FUNCTIONAL': 'PBE',
    'BASIS_SET_FILE_NAME': 'GTH_BASIS_SETS',
    'POTENTIAL_FILE_NAME': 'POTENTIAL',
    'MGRID_CUTOFF': 450.0,
    'MGRID_REL_CUTOFF': 75.0,
    'SCF_MAX_SCF': 200,
    'MO_CUBES_NHOMO': 6,
    'MO_CUBES_NLUMO': 6,
    'MO_CUBES_STRIDE': 5,
}
# default cp2k:qs output path
cp2k_out=f'cp2k_output/{CP2K_QS_GGA_parameters["PROJECT_NAME"]}.log'

In [ ]:
# read a single molecule xyz file
mymol=single_molecule(config['single_mol_xyz'])
# prepare CP2K:QS input file for single molecule
mymol.prep_cp2k_single(CP2K_QS_GGA_parameters,config['label'],config['cp2k_template'],config['basis'],config['potential'])
print(f'Note: make sure CP2K:QS output gets in "{cp2k_out}"')

Generated output/6T/6T.inp
Note: make sure CP2K:QS output gets in "cp2k_output/6T.log"


In [ ]:
# provided we carried out the DFT calculation and placed the CP2K output in the appropriate directory,
# read CP2K:QS output and retrieve MO info
mymol.get_cp2k_info(config['MO'],cp2k_out,config['cp2k_basis_file'],config['basis'])





In [ ]:


# STO init and overlap matrix calculation
mymol.initialize_STOs(STO_proj_dict)
# carry out GTO->STO projection
mymol.project()

In [ ]:
# store AOM coefficients
mymol.save_AOM(config['label'])

In [ ]:
# projection completeness
mymol.orb_compl_dict

{73: 0.9852359470457339}

In [ ]:
# cube file preview; optional
mymol.cube_preview(STO_proj_dict,f'output/{config["label"]}/{config["label"]}_{config["MO"]}.cube')

## PRINT PYCOEFF IN INCLUDE FORMAT

In [ ]:
dict_anumber = {
    "C": 6,
    "H": 6,
    "S": 16,
    "N": 7,
    "O": 8,
}

names = mymol.species
coeff = mymol.AOM_pi_coeffs

for index,n in enumerate(mymol.species):
    print(names[index], dict_anumber[names[index]], "1", "0.00000", coeff[index])

S 16 1 0.00000 -0.11355184961620941
S 16 1 0.00000 0.07209018120225767
S 16 1 0.00000 -0.024021626898581817
C 6 1 0.00000 -0.20292483827172533
C 6 1 0.00000 -0.08403943948850537
C 6 1 0.00000 0.18399499902888158
C 6 1 0.00000 -0.11158509986888168
C 6 1 0.00000 -0.26870199084476887
C 6 1 0.00000 0.16319301496570762
C 6 1 0.00000 -0.229034499475736
C 6 1 0.00000 0.20150009182721362
C 6 1 0.00000 0.280855885864112
C 6 1 0.00000 -0.21654153337614132
C 6 1 0.00000 0.23882826759996054
C 6 1 0.00000 -0.25776433538605004
H 6 1 0.00000 0.0
H 6 1 0.00000 0.0
H 6 1 0.00000 0.0
H 6 1 0.00000 0.0
H 6 1 0.00000 0.0
H 6 1 0.00000 0.0
H 6 1 0.00000 0.0
C 6 1 0.00000 0.25776433538605004
S 16 1 0.00000 -0.024021626899589282
C 6 1 0.00000 -0.280855885864112
C 6 1 0.00000 -0.20150009182721362
S 16 1 0.00000 0.07209018120326514
C 6 1 0.00000 0.26870199084476887
C 6 1 0.00000 0.11158509986888168
S 16 1 0.00000 -0.11355184961620941
C 6 1 0.00000 0.20292483827172533
C 6 1 0.00000 0.08403943948850537
C 6 1 0.0

### OVERLAP CALCULATIONS

In [ ]:
# define STO parameters for AOM calculations
AOM_dict={
    'H':{'STOs':1  ,'1s':1.0000},
    'C':{'STOs':1+3,'2s':1.6083,'2p':1.385600},
    'N':{'STOs':1+3,'2s':1.9237,'2p':1.617102},
    'O':{'STOs':1+3,'2s':2.2458,'2p':1.505135},
    'F':{'STOs':1+3,'2s':2.5638,'2p':1.665190},
    'S':{'STOs':1+3,'3s':2.1223,'3p':1.641119},
}

In [ ]:
# locate dimers: define the directory with dimers
# suggested file structure: create a directory with the molecule name in 'dimers'
# and use the following naming for each dimer: <name>_<No>_<comment>.xyz
dimer_dir='dimers/OT6_crystal_pairs/'
# define the AOM files for both dimer fragments
frag1_AOM_file='output/6T/AOM_COEFF.dat'
frag2_AOM_file='output/6T/AOM_COEFF.dat'
# define MOs
frag1_MO=73
frag2_MO=73
#--------------------------------------------------------------------------------



#Sab_array=[]
#for file in sorted([i for i in os.listdir(dimer_dir) if i.endswith('.xyz')==True]):
#    print(file)
#    dimer_xyz_file=f'{dimer_dir}/{file}'
#    val=Sab(dimer_xyz_file,frag1_AOM_file,frag2_AOM_file,frag1_MO,frag2_MO,AOM_dict)
#    Sab_array.append(val)
#    print(f'{file} Sab: {val}')

print(os.listdir(dimer_dir))
name=["/P_pair.xyz",  "/Px_pair.xyz",  "/T1_pair.xyz"]
#name=["/P.xyz",  "/T1_close.xyz",  "/T2_close.xyz"]

Sab_array=[]
for file in name:
    #print(file)
    dimer_xyz_file = dimer_dir + file
    print(dimer_xyz_file)

    val=Sab(dimer_xyz_file,frag1_AOM_file,frag2_AOM_file,frag1_MO,frag2_MO,AOM_dict)
    Sab_array.append(val)
    print(f'{file} Sab: {val}')

['crystal_cell.xyz', 'Px_pair.xyz', 'P_pair.xyz', 'T1_pair.xyz']
dimers/OT6_crystal_pairs//P_pair.xyz
/P_pair.xyz Sab: 0.00028573799627411756
dimers/OT6_crystal_pairs//Px_pair.xyz
/Px_pair.xyz Sab: -6.747775717861905e-06
dimers/OT6_crystal_pairs//T1_pair.xyz
/T1_pair.xyz Sab: 0.004842354740900467


In [ ]:
# define STO parameters for AOM calculations
AOM_dict={
    'H':{'STOs':1  ,'1s':1.0000},
    'C':{'STOs':1+3,'2s':1.6083,'2p':1.385600},
    'N':{'STOs':1+3,'2s':1.9237,'2p':1.617102},
    'O':{'STOs':1+3,'2s':2.2458,'2p':1.505135},
    'F':{'STOs':1+3,'2s':2.5638,'2p':1.665190},
    'S':{'STOs':1+3,'3s':2.1223,'3p':1.641119},
}

In [ ]:
# locate dimers: define the directory with dimers
# suggested file structure: create a directory with the molecule name in 'dimers'
# and use the following naming for each dimer: <name>_<No>_<comment>.xyz
dimer_dir='dimers/'
# define the AOM files for both dimer fragments
frag1_AOM_file='output/6T/AOM_COEFF.dat'
frag2_AOM_file='output/PDI/HOMO_AOM_COEFF.dat'
# define MOs
frag1_MO=73
frag2_MO=70
#--------------------------------------------------------------------------------



#Sab_array=[]
#for file in sorted([i for i in os.listdir(dimer_dir) if i.endswith('.xyz')==True]):
#    print(file)
#    dimer_xyz_file=f'{dimer_dir}/{file}'
#    val=Sab(dimer_xyz_file,frag1_AOM_file,frag2_AOM_file,frag1_MO,frag2_MO,AOM_dict)
#    Sab_array.append(val)
#    print(f'{file} Sab: {val}')

print(os.listdir(dimer_dir))
name=["/interface_dimer.xyz", "interface_problem_stuc283-305.xyz", "interface_problem_stuc283-305-step1.xyz", "interface_problem_stuc283-329.xyz", "interface_problem_stuc283-305-allameno2.xyz" ]
#name=["/P.xyz",  "/T1_close.xyz",  "/T2_close.xyz",]

Sab_array=[]
for file in name:
    #print(file)
    dimer_xyz_file = dimer_dir + file
    print(dimer_xyz_file)

    val=Sab(dimer_xyz_file,frag1_AOM_file,frag2_AOM_file,frag1_MO,frag2_MO,AOM_dict)
    Sab_array.append(val)
    print(f'{file} Sab: {val}')

['interface_dimer.xyz', 'interface_problem_stuc283-305-allameno2.xyz', 'interface_problem_stuc283-305-step1.xyz', 'interface_problem_stuc283-305.xyz', 'interface_problem_stuc283-329.xyz', 'OT6_crystal_pairs', 'PDI-Crystal_pairs', 'pos-init.xyz', 'run-pos-1.xyz', 'step48.xyz', 'TTF']
dimers//interface_dimer.xyz
/interface_dimer.xyz Sab: -0.003362012342808487
dimers/interface_problem_stuc283-305.xyz
interface_problem_stuc283-305.xyz Sab: -0.0033557260345155582
dimers/interface_problem_stuc283-305-step1.xyz
interface_problem_stuc283-305-step1.xyz Sab: -0.0033556759358456738
dimers/interface_problem_stuc283-329.xyz
interface_problem_stuc283-329.xyz Sab: -3.8669372023439637e-10
dimers/interface_problem_stuc283-305-allameno2.xyz
interface_problem_stuc283-305-allameno2.xyz Sab: -0.0032993313452111006
